In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt
from itertools import chain, combinations
from pulp import LpStatus, LpStatusInfeasible
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value
import networkx as nx
from pulp import GUROBI



num = 9  # which instance out of the 10 we use

def generate_random_graph(n, p):
    """Generates a random NetworkX graph with n vertices and edge probability p."""
    G = nx.Graph()
    G.add_nodes_from(range(n))
    
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < p:
                G.add_edge(i, j)
    
    return G




def generate_random_digraph(n, p):
    """Generates a random NetworkX graph with n vertices and edge probability p."""
    G = nx.DiGraph()
    G.add_nodes_from(range(n))
    
    for i in range(n):
        for j in range(n):
            if random.random() < p:
                G.add_edge(i, j) 
    return G


def generate_typed_graph(t, n, p=0.3, q=1, seed=None):
    """
    Generates a directed graph with n nodes, each assigned a random type from 0 to t-1.
    For every pair of types (t1, t2), if a random number >= 0.5, then for every u of type t1
    and v of type t2, an arc (u, v) is added with probability p.

    Parameters:
        t (int): Number of types.
        n (int): Number of nodes.
        p (float): Edge probability between compatible type pairs.
        q: unused, present to match your input signature.
        seed (int, optional): Random seed for reproducibility.

    Returns:
        G (nx.DiGraph): Generated graph with node attribute "type".
    """
    if seed is not None:
        random.seed(seed)

    G = nx.DiGraph()

    # Assign types
    types = {i: random.randint(0, t - 1) for i in range(n)}
    nx.set_node_attributes(G, types, "type")
    G.add_nodes_from(types.keys())

    # Decide compatibility for each type pair
    compatibility = {
        (t1, t2): random.random() >= p
        for t1 in range(t)
        for t2 in range(t)
    }

    # Add edges based on type compatibility and edge probability
    for u in range(n):
        for v in range(n):
            if u == v:
                continue
            type_u = types[u]
            type_v = types[v]
            if compatibility[(type_u, type_v)] and random.random() < q:
                G.add_edge(u, v)

    return G

def draw_graph(adj_list):
    """Draws the graph using NetworkX."""
    G = nx.Graph()
    for node, neighbors in adj_list.items():
        for neighbor in neighbors:
            G.add_edge(node, neighbor)
    
    nx.draw(G, with_labels=True, node_color='lightblue', edge_color='gray')
    plt.show()

def parse_xml_and_create_graph(file_path, with_weights=False):
    """Parses the XML file and creates a directed graph."""
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    # Extract donor data
    donors = []
    for entry in root.findall("entry"):
        donor_id = int(entry.get("donor_id"))
        matches = []
        matches_element = entry.find("matches")
        if matches_element is not None:
            for match in matches_element.findall("match"):
                recipient = int(match.find("recipient").text)
                score = int(match.find("score").text) if with_weights else 1
                matches.append((recipient, score))
        donors.append({"donor_id": donor_id, "matches": matches})
    
    # Create directed graph
    G = nx.DiGraph()
    for donor in donors:
        donor_id = donor["donor_id"]
        G.add_node(donor_id)
        for recipient, score in donor["matches"]:
            G.add_edge(donor_id, recipient, weight=score)
    if with_weights:
        edge_labels = {(u, v): d['weight'] for u, v, d in G.edges(data=True)}
    return G

def draw_directed_graph(G):
    """Draws the directed graph."""
    plt.figure(figsize=(10, 6))
    pos = nx.spring_layout(G)
    edges = G.edges(data=True)
    
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', arrows=True)
    
    plt.show()

def partition_vertices(G, k):
    """Partitions the vertices into k random countries."""
    nodes = list(G.nodes())
    random.shuffle(nodes)
    partitions = {node: i % k for i, node in enumerate(nodes)}
    return partitions

def partition_vertices_uneven(G, k ,l):
    """Partitions the vertices into k large and l small countries."""
    nodes = list(G.nodes())
    random.shuffle(nodes)
    partitions = {node: i % k for i, node in enumerate(nodes[:len(nodes)*2//3])}
    for j in range(len(nodes)*2//3, len(nodes)):
        partitions[nodes[j]] = j % l + k
    return partitions

def create_symmetric_undirected_graph(G):
    """Creates an undirected graph where an edge exists if and only if both directed edges exist in G."""
    UG = nx.Graph()
    for u, v in G.edges():
        if G.has_edge(v, u):
            UG.add_edge(u, v)
    return UG

def to_directed(G):
    D = nx.DiGraph()
    D.add_nodes_from(G.nodes())
    for u,v in G.edges():
        D.add_edges_from([(u,v),(v,u)])
    return D

def find_maximum_matching(UG):
    """Finds a maximum size matching in UG and prints it."""
    matching = nx.max_weight_matching(UG, maxcardinality=True)
    print("Maximum Matching:", matching)
    return matching

def powerset(iterable):
    """Returns all non-empty subsets of an iterable."""
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s) + 1))

#FOR MATCHING CASE ONLY
def check_core_stability(UG, M, partitions):
    """Checks if the matching M is in the core by verifying against blocking coalitions."""
    # Count matched vertices in each country
    country_match_count = {i: 0 for i in set(partitions.values())}
    for u, v in M:
        country_match_count[partitions[u]] += 1
        country_match_count[partitions[v]] += 1
    
    for coalition in powerset(set(partitions.values())):
        coalition_vertices = {v for v, c in partitions.items() if c in coalition}
        subgraph = UG.subgraph(coalition_vertices).copy()
        sub_matching = nx.max_weight_matching(subgraph, maxcardinality=True)
        
        matched_count_coalition = sum(2 for u, v in sub_matching if u in coalition_vertices and v in coalition_vertices)
        matched_count_M = sum(1 for u, v in M if partitions[u] in coalition or partitions[v] in coalition) + sum(1 for u, v in M if partitions[u] in coalition and partitions[v] in coalition)
        
        if matched_count_coalition > matched_count_M:
            print("Not in core. Blocking coalition:", coalition, sub_matching)
            return "not in core", coalition
    
    print("Matching is in the core.")
    return "in the core", None

# FOR MATCHING CASE ONLY!
def check_weak_core_stability(UG, M, partitions):
    """Checks weak core stability by adding dummy vertices and verifying perfect matchings."""
    country_match_count = {i: 0 for i in set(partitions.values())}
    country_total_count = {i: 0 for i in set(partitions.values())}
    
    for v, c in partitions.items():
        country_total_count[c] += 1
    
    for u, v in M:
        country_match_count[partitions[u]] += 1
        country_match_count[partitions[v]] += 1
    
    for coalition in powerset(set(partitions.values())):
        coalition_vertices = {v for v, c in partitions.items() if c in coalition}
        subgraph = UG.subgraph(coalition_vertices).copy()
        
        dummy_vertices = []
        chance = True
        for country in coalition:
            needed_dummies = country_total_count[country] - country_match_count[country] - 1
            if needed_dummies < 0:
                chance = False
            for i in range(needed_dummies):
                dummy_node = f"dummy_{country}_{i}"
                dummy_vertices.append(dummy_node)
                subgraph.add_node(dummy_node)
                for v in [n for n, c in partitions.items() if c == country]:
                    subgraph.add_edge(dummy_node, v)
        for u, v in subgraph.edges():
            if u in coalition_vertices and v in coalition_vertices:
                subgraph[u][v]['weight'] = 2
            else:
                subgraph[u][v]['weight'] = 1
        
        sub_matching = nx.max_weight_matching(subgraph, maxcardinality=False, weight='weight')
        total_weight = sum(subgraph[u][v]['weight'] for u, v in sub_matching)
        total_real_vertices = sum(country_total_count[c] for c in coalition)
        
        if total_weight == total_real_vertices and chance:  
            return "Not in weak core", coalition, sub_matching, [country_match_count[c] for c in coalition], [country_total_count[c] for c in coalition]
    
    return "Matching is in the weak core."

""" General cycle length (max_len)"""

GRB_WLSACCESSID = "089eb4e7-7705-4eec-b0c9-88a623f92759"
GRB_WLSSECRET = "f53c2bd0-9f7a-4232-9146-b32977468009"
GRB_LICENSEID = "2644044"

def create_integer_program(G, weights = None, max_len = float('inf'), PRE_ADD_PATHS = False):
    """Create an ILP for maximum directed cycle packing."""
    prob = LpProblem("Maximum_Directed_Matching", LpMaximize)
    
    # Binary variables for edges
    x = { (u, v): LpVariable(f"x_{u}_{v}", cat='Binary') for u, v in G.edges() }

    """ NEW PART """
    if max_len != float('inf') and PRE_ADD_PATHS:
        long_paths = find_long_paths(G, G.edges(), max_len)
        for path in long_paths:
            prob += lpSum(x[u, v] for u, v in zip(path, path[1:])) <= max_len - 1
    """ END OF NEW PART"""
    
    # Flow conservation constraints
    for v in G.nodes():
        prob += lpSum(x[u, v] for u in G.predecessors(v) if (u, v) in x) == lpSum(x[v, w] for w in G.successors(v) if (v, w) in x)
    
    # At most one outgoing edge per node
    for v in G.nodes():
        prob += lpSum(x[v, w] for w in G.successors(v) if (v, w) in x) <= 1
    if weights:
        prob += lpSum(x[u, v]*weights[(u,v)] for u, v in G.edges())
    else:# Objective: maximize the number of selected edges
        prob += lpSum(x[u, v] for u, v in G.edges())
    return prob, x

def extract_selected_edges(G, x):
    """Extracts the set of edges included in the current optimal solution."""
    return [(u, v) for (u, v), var in x.items() if value(var) > 0.5]

def find_long_paths(G, selected_edges, max_len):
    """Returns all directed paths in selected_edges with exactly max_len edges."""
    # Build subgraph from selected edges
    subgraph = nx.DiGraph()
    subgraph.add_edges_from(selected_edges)

    exact_len_paths = []

    for source in subgraph.nodes():
        # Use DFS stack: (current_node, path_so_far)
        stack = [(source, [source])]
        while stack:
            current, path = stack.pop()
            if len(path) - 1 == max_len:
                exact_len_paths.append(path)
            elif len(path) - 1 < max_len:
                for neighbor in subgraph.successors(current):
                    if neighbor not in path:  # avoid cycles
                        stack.append((neighbor, path + [neighbor]))

    return exact_len_paths

def find_maxL_packing(G, max_len = float('inf')):
    """Solve the ILP dynamically adding violated path constraints."""
    prob, x = create_integer_program(G, max_len = max_len)

    while True:
        # Solve the ILP
        prob.solve(GUROBI(msg=False))

        if max_len == float('inf'):
            break
        # Extract selected edges
        selected_edges = extract_selected_edges(G, x)
        
        # Find violated constraints (paths too long)
        long_paths = find_long_paths(G, selected_edges, max_len)
        
        if not long_paths:
            break  # No violations, terminate
        
        # Add new constraints for found paths
        for path in long_paths:
            prob += lpSum(x[u, v] for u, v in zip(path, path[1:])) <= max_len - 1
    
    return extract_selected_edges(G, x)  # Return final solution


def check_TU_core_stability_general(G, C, partitions, improvement = 1, max_len = float('inf')):
    """Checks weak core stability."""
    original_nodes = set(partitions.keys())
    country_match_count = {i: 0 for i in set(partitions.values())}
    country_total_count = {i: 0 for i in set(partitions.values())}

    num_IR_violations = 0
    num_core_violations = 0
    
    for v, c in partitions.items():
        country_total_count[c] += 1
    
    for (u, v) in C:
        if v in original_nodes:
            country_match_count[partitions[v]] += 1

    #print(country_match_count)
    
    for coalition in powerset(set(partitions.values())):
        coalition_vertices = {v for v, c in partitions.items() if c in coalition}
        subgraph = G.subgraph(coalition_vertices).copy()

        prob , x  = create_integer_program(subgraph, max_len = max_len)
        for country in coalition:
            prob += lpSum(x[u,v] for u,v in subgraph.edges() ) >= np.sum([country_match_count[country] for country in coalition]) + improvement
        No_Sol = False
        while True:
        # Solve the ILP
            prob.solve(GUROBI(msg=False))

            if prob.status == LpStatusInfeasible:  
            # If infeasible, move to the next coalition
                No_Sol = True
                break  
            
            if max_len == float('inf'):
                break
     # Extract selected edges
            selected_edges = extract_selected_edges(G, x)
        
        # Find violated constraints (paths too long)
            long_paths = find_long_paths(G, selected_edges, max_len)
        
            if not long_paths:
                break  # No violations, terminate
        
        # Add new constraints for found paths
            for path in long_paths:
                prob += lpSum(x[u, v] for u, v in zip(path, path[1:])) <= max_len - 1

        if No_Sol == False:
            sub_solution = extract_selected_edges(G, x)
            
            total_real_vertices = sum(country_total_count[c] for c in coalition)
            num_core_violations += 1
            if len(coalition) == 1:
                num_IR_violations += 1

    if num_core_violations > 0:
        return "Not in TU core", coalition, sub_solution, num_IR_violations, num_core_violations
    
    return "Solution is in the TU core." , None, None, None, None



def TU_core_heuristic(G, partition, improvement = 1, max_len=float('inf')):

    weights = {(u,v) : len(G.nodes()) for u,v in G.edges}
    prob, x = create_integer_program(G, weights = weights, max_len = max_len)

    not_TU_core = True
    i=0
    while not_TU_core and i<=100:
        i += 1
      # Solve the ILP
        while True:
            prob.solve(GUROBI(msg=False))
    
            if max_len == float('inf'):
                break
            # Extract selected edges
            selected_edges = extract_selected_edges(G, x)
            
            # Find violated constraints (paths too long)
            long_paths = find_long_paths(G, selected_edges, max_len)
            
            if not long_paths:
                break  # No violations, terminate
            
            # Add new constraints for found paths
            for path in long_paths:
                prob += lpSum(x[u, v] for u, v in zip(path, path[1:])) <= max_len - 1
        
        solution = extract_selected_edges(G, x) 
        _, coalition , _ , num_IR_violations , num_core_violations = check_TU_core_stability_general(G, solution, partition, improvement = improvement, max_len= max_len)
        if i == 1:
            print(f"Violations: IR:{num_IR_violations}, CORE: {num_core_violations}")
        if coalition == None:
            return solution, i 
        
        else: 
            for u,v in G.edges:
                if v in coalition:
                    weights[(u,v)] += 1
            prob += lpSum(x[u, v]*weights[(u,v)] for u, v in G.edges())
    return None , i 
    
def near_feasible_TU_core_heuristic(G, partition, max_len = float('inf')):
    improvement = 0
    solution, i = TU_core_heuristic(G, partition, improvement = improvement + 1, max_len = max_len)
    while not solution:
        solution, i = TU_core_heuristic(G, partition, improvement = improvement + 1, max_len = max_len)
        improvement += 1
        
    return solution, improvement , i

def weak_core_heuristic(G, partition, improvement = 1, max_len=float('inf')):

    weights = {(u,v) : len(G.nodes()) for u,v in G.edges}
    prob, x = create_integer_program(G, weights = weights, max_len = max_len)

    not_weak_core = True
    i=0
    while not_weak_core and i<=100:
        i += 1
        # Solve the ILP
        while True:
            prob.solve(GUROBI(msg=False))
    
            if max_len == float('inf'):
                break
            # Extract selected edges
            selected_edges = extract_selected_edges(G, x)
            
            # Find violated constraints (paths too long)
            long_paths = find_long_paths(G, selected_edges, max_len)
            
            if not long_paths:
                break  # No violations, terminate
            
            # Add new constraints for found paths
            for path in long_paths:
                prob += lpSum(x[u, v] for u, v in zip(path, path[1:])) <= max_len - 1
        
        solution = extract_selected_edges(G, x) 
        _, coalition , _ , num_IR_violations , num_core_violations = check_weak_core_stability_general(G, solution, partition, improvement = improvement, max_len= max_len)
        if i == 1:
            print(f"Violations: IR: {num_IR_violations}, CORE: {num_core_violations}, Coalition: {coalition}")
        if coalition == None:
            return solution , i
        else: 
            for u,v in G.edges:
                if v in coalition:
                    weights[(u,v)] += 1
            prob += lpSum(x[u, v]*weights[(u,v)] for u, v in G.edges())
    return None, i
    
    

def near_feasible_weak_core_heuristic(G, partition, max_len = float('inf'), total = False):
    
    if total:
        D = G.copy()
        solution = None
        total_donors = 0
        while not solution:
            solution, i = weak_core_heuristic(D, partition, improvement = 1 , max_len = max_len)
            new_node = max(D.nodes, default=-1) + 1  # Ensure unique node ID
            D.add_node(new_node)
            for v in D.nodes:
                if v != new_node:
                    D.add_edge(new_node, v)  # Out-arc
                    D.add_edge(v, new_node)  # In-arc
            total_donors += 1
        return solution, total_donors - 1, i
    else:
        solution = None
        improvement = 0
        while not solution:
            solution, i = weak_core_heuristic(G, partition, improvement = improvement + 1, max_len = max_len)
            improvement += 1
        return solution, improvement - 1, i


def check_weak_core_stability_general(G, C, partitions, improvement = 1, max_len = float('inf')):
    """Checks weak core stability."""
    original_nodes = set(partitions.keys())
    country_match_count = {i: 0 for i in set(partitions.values())}
    country_total_count = {i: 0 for i in set(partitions.values())}
    num_IR_violations = 0
    num_core_violations = 0
    blocking_coalition = None
    
    for v, c in partitions.items():
        country_total_count[c] += 1
    
    for (u, v) in C:
        if v in original_nodes:
            country_match_count[partitions[v]] += 1

    #print(country_match_count)
    
    for coalition in powerset(set(partitions.values())):
        coalition_vertices = {v for v, c in partitions.items() if c in coalition}
        subgraph = G.subgraph(coalition_vertices).copy()

        prob , x  = create_integer_program(subgraph, max_len = max_len)
        for country in coalition:
            prob += lpSum(x[u,v] for u,v in subgraph.edges() if partitions[v] == country) >= country_match_count[country] + improvement
        No_Sol = False
        while True:
        # Solve the ILP
            prob.solve(GUROBI(msg=False))

            if prob.status == LpStatusInfeasible:  
            # If infeasible, move to the next coalition
                No_Sol = True
                break  
            
            if max_len == float('inf'):
                break
        # Extract selected edges
            selected_edges = extract_selected_edges(G, x)
        
        # Find violated constraints (paths too long)
            long_paths = find_long_paths(G, selected_edges, max_len)
        
            if not long_paths:
                break  # No violations, terminate
        
        # Add new constraints for found paths
            for path in long_paths:
                prob += lpSum(x[u, v] for u, v in zip(path, path[1:])) <= max_len - 1
        #print(coalition)
        if No_Sol == False:
            sub_solution = extract_selected_edges(G, x)
            blocking_coalition = coalition
          #  total_real_vertices = sum(country_total_count[c] for c in coalition)
            num_core_violations += 1
            if len(coalition) == 1:
                num_IR_violations += 1
                
            

    if num_core_violations > 0:
        return "Not in weak core", blocking_coalition, sub_solution, num_IR_violations, num_core_violations
    
    return "Solution is in the weak core." , None, None, None, None


def find_TU_core(G, partitions, improvement = 0, max_len = float('inf')):
    Big_prob, Big_x = create_integer_program(G, max_len = max_len)
    for coalition in powerset(set(partitions.values())):
        coalition_vertices = {v for v, c in partitions.items() if c in coalition}
        subgraph = G.subgraph(coalition_vertices).copy()

        val_coalition = len(find_maxL_packing(subgraph, max_len))
        Big_prob += lpSum(Big_x[u,v] for u,v in G.edges() if v in coalition_vertices ) >= val_coalition - improvement
    while True:
        # Solve the ILP
        Big_prob.solve(GUROBI(msg=False))
        
        if Big_prob.status == LpStatusInfeasible:  
            return None
        if max_len == float('inf'):
            return extract_selected_edges(G, Big_x)
        # Extract selected edges
        selected_edges = extract_selected_edges(G, Big_x)
        
        # Find violated constraints (paths too long)
        long_paths = find_long_paths(G, selected_edges, max_len)
        
        if not long_paths:
            break  # No violations, terminate
        
        # Add new constraints for found paths
        for path in long_paths:
            Big_prob += lpSum(Big_x[u, v] for u, v in zip(path, path[1:])) <= max_len - 1

    return extract_selected_edges(G, Big_x)  # Return final solution


def EXACT_near_feasible_TU_core(G, partitions, max_len = float('inf'), total = False):
    if total:
        D = G.copy()
        solution = None
        total_donors = 0
        while not solution:
            solution = find_TU_core(D, partitions, improvement = 0 , max_len = max_len)
            new_node = max(D.nodes, default=-1) + 1  # Ensure unique node ID
             
            D.add_node(new_node)
            for v in D.nodes:
                if v != new_node:
                    D.add_edge(new_node, v)  # Out-arc
                    D.add_edge(v, new_node)  # In-arc
            total_donors += 1
        return solution, total_donors -1
    else:
        Big_prob, Big_x = create_integer_program(G, max_len = max_len)
        coalition_vals = {}
        improvement = 0
        LONG_paths = []
        for coalition in powerset(set(partitions.values())):
            coalition_vertices = {v for v, c in partitions.items() if c in coalition}
            subgraph = G.subgraph(coalition_vertices).copy()
            
            coalition_vals[coalition] = len(find_maxL_packing(subgraph, max_len))
        
            Big_prob += lpSum(Big_x[u,v] for u,v in G.edges() if v in coalition_vertices ) >= coalition_vals[coalition] - improvement
        #print(coalition_vals)
        #print("got_to_IP_solving")
        while True:
        # Solve the ILP
            Big_prob.solve(GUROBI(msg=False))
            
            if Big_prob.status == LpStatusInfeasible:  
                #solution = None
                improvement += 1
                #print(improvement)
                Big_prob, Big_x = create_integer_program(G, max_len = max_len)
                for coalition in powerset(set(partitions.values())):
                    coalition_vertices = {v for v, c in partitions.items() if c in coalition}
                    subgraph = G.subgraph(coalition_vertices).copy()
                    Big_prob += lpSum(Big_x[u,v] for u,v in G.edges() if v in coalition_vertices) >= coalition_vals[coalition] - improvement
                for path in LONG_paths:
                    Big_prob += lpSum(Big_x[u, v] for u, v in zip(path, path[1:])) <= max_len - 1
            elif max_len == float('inf'):
                return extract_selected_edges(G, Big_x), improvement
            else:
                selected_edges = extract_selected_edges(G, Big_x)
                #print(selected_edges)
                # Find violated constraints (paths too long)
                long_paths = find_long_paths(G, selected_edges, max_len)
                LONG_paths += long_paths
                
                if not long_paths:
                    return selected_edges, improvement  # No violations, terminate
                
                # Add new constraints for found paths
                for path in long_paths:
                    Big_prob += lpSum(Big_x[u, v] for u, v in zip(path, path[1:])) <= max_len - 1
           
       
     


In [ ]:
""" 5 player, KEP """ 

In [16]:
import concurrent.futures
import time
import random
import networkx as nx

solutions_len2_k5_WC = []
solutions_len2_k5_TUC = []
solutions_len3_k5_TUC = []
solutions_len3_k5_WC = []

k = 5  # Number of countries

for num in range(20):
    file_path = f"/Users/User/Downloads/KEP_instances/genxml-{num}.xml"
    
    
    G = parse_xml_and_create_graph(file_path, with_weights=False)
    mapping = dict(zip(G.nodes, random.sample(range(len(G.nodes)), len(G.nodes))))
    G_permuted = nx.relabel_nodes(G, mapping)
    G = G_permuted.subgraph({i for i in range(100)} )
    
    partitions = partition_vertices(G, k)

    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 2, total = True)
    solutions_len2_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=2: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len2_k5_TUC.append(0)
        print(f"TU CORE, max_len=2: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 2)
        solutions_len2_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=2: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2, total = True)
        solutions_len2_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors <= 1:
            solutions_len2_k5_WC.append(0)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2)
            solutions_len2_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")
            
    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 3, total = True)
    solutions_len3_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=3: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len3_k5_TUC.append(0)
        print(f"TU CORE, max_len=3: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 3)
        solutions_len3_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=3: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3, total = True)
        solutions_len3_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors <=1:
            solutions_len3_k5_WC.append(0)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3)
            solutions_len3_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")

    


TU CORE max_len=2: for instance 0: we need 0  donors.
TU CORE, max_len=2: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 0: we need 0  donors.
TU CORE, max_len=3: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=3: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=3: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=2: for instance 1: we need 0  donors.
TU CORE, max_len=2: for instance 1: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 1: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 1: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 1: we need 0  donors.
TU CORE, max_len=3: for instance 1: we need 0 improvement in constraints.
WEAK CORE, max_len=3: for instance 1: we need 0 donors. STEPS:

In [ ]:
""" 6 country, KEP """

In [18]:
solutions_len2_k5_WC = []
solutions_len2_k5_TUC = []
solutions_len3_k5_TUC = []
solutions_len3_k5_WC = []

k = 6  # Number of countries

for num in range(20):
    file_path = f"/Users/User/Downloads/KEP_instances/genxml-{num}.xml"
    
    
    G = parse_xml_and_create_graph(file_path, with_weights=False)
    mapping = dict(zip(G.nodes, random.sample(range(len(G.nodes)), len(G.nodes))))
    G_permuted = nx.relabel_nodes(G, mapping)
    G = G_permuted.subgraph({i for i in range(130)} )
    
    partitions = partition_vertices(G, k)

    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 2, total = True)
    solutions_len2_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=2: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len2_k5_TUC.append(0)
        print(f"TU CORE, max_len=2: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 2)
        solutions_len2_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=2: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2, total = True)
        solutions_len2_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors <= 1:
            solutions_len2_k5_WC.append(0)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2)
            solutions_len2_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")
    
    G = parse_xml_and_create_graph(file_path, with_weights=False)
    mapping = dict(zip(G.nodes, random.sample(range(len(G.nodes)), len(G.nodes))))
    G = G_permuted.subgraph({i for i in range(100)} )
    
    partitions = partition_vertices(G, k)
    
    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 3, total = True)
    solutions_len3_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=3: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len3_k5_TUC.append(0)
        print(f"TU CORE, max_len=3: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 3)
        solutions_len3_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=3: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3, total = True)
        solutions_len3_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors <= 1:
            solutions_len3_k5_WC.append(0)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3)
            solutions_len3_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")

    


TU CORE max_len=2: for instance 0: we need 0  donors.
TU CORE, max_len=2: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 0: we need 0  donors.
TU CORE, max_len=3: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=3: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=3: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=2: for instance 1: we need 0  donors.
TU CORE, max_len=2: for instance 1: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 1: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 1: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 1: we need 1  donors.
TU CORE, max_len=3: for instance 1: we need 1 improvement in constraints.
Violations: IR: None, CORE: None, Coalition: None
WEAK CORE, m

In [2]:
solutions_len2_k5_WC = []
solutions_len2_k5_TUC = []
solutions_len3_k5_TUC = []
solutions_len3_k5_WC = []

k = 6  # Number of countries

for num in range(20):
    file_path = f"/Users/User/Downloads/KEP_instances/genxml-{num}.xml"
    
    
    G = parse_xml_and_create_graph(file_path, with_weights=False)
    mapping = dict(zip(G.nodes, random.sample(range(len(G.nodes)), len(G.nodes))))
    G_permuted = nx.relabel_nodes(G, mapping)
    G = G_permuted.subgraph({i for i in range(130)} )
    
    partitions = partition_vertices(G, k)

    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 2, total = True)
    solutions_len2_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=2: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len2_k5_TUC.append(0)
        print(f"TU CORE, max_len=2: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 2)
        solutions_len2_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=2: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2, total = True)
        solutions_len2_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors <= 1:
            solutions_len2_k5_WC.append(0)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2)
            solutions_len2_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")
    
    G = parse_xml_and_create_graph(file_path, with_weights=False)
    mapping = dict(zip(G.nodes, random.sample(range(len(G.nodes)), len(G.nodes))))
    G = G_permuted.subgraph({i for i in range(100)} )
    
    partitions = partition_vertices(G, k)
    
    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 3, total = True)
    solutions_len3_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=3: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len3_k5_TUC.append(0)
        print(f"TU CORE, max_len=3: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 3)
        solutions_len3_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=3: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3, total = True)
        solutions_len3_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors <= 1:
            solutions_len3_k5_WC.append(0)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3)
            solutions_len3_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")

    


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2644044
Academic license 2644044 - for non-commercial use only - registered to cs___@student.elte.hu
TU CORE max_len=2: for instance 0: we need 0  donors.
TU CORE, max_len=2: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 0: we need 0  donors.
TU CORE, max_len=3: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=3: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=3: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=2: for instance 1: we need 1  donors.
TU CORE, max_len=2: for instance 1: we need 1 improvement in constraints.
Violations: IR: 1, CORE: 1, Coalition: (2,)


C:\Users\User\anaconda3\Lib\site-packages\pulp\pulp.py:1780: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Violations: IR: None, CORE: None, Coalition: None
WEAK CORE, max_len=2: for instance 1: we need 1 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 1: we need 1 donors per country. STEPS: 1
TU CORE max_len=3: for instance 1: we need 1  donors.
TU CORE, max_len=3: for instance 1: we need 1 improvement in constraints.
Violations: IR: None, CORE: None, Coalition: None
WEAK CORE, max_len=3: for instance 1: we need 0 donors. STEPS: 1
WEAK CORE, max_len=3: for instance 1: we need 0 donors per country. STEPS: 1
TU CORE max_len=2: for instance 2: we need 0  donors.
TU CORE, max_len=2: for instance 2: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 2: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 2: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 2: we need 0  donors.
TU CORE, max_len=3: for instance 2: we need 0 improvement in constraints.
WEAK CORE, max_len=3: for instance 2: we need 0 donors. STEPS: 1
WEAK CORE, max_len=3: for

In [ ]:
""" k= 4, KEP """

In [3]:
solutions_len2_k5_WC = []
solutions_len2_k5_TUC = []
solutions_len3_k5_TUC = []
solutions_len3_k5_WC = []

k = 4  # Number of countries

for num in range(20):
    file_path = f"/Users/User/Downloads/KEP_instances/genxml-{num}.xml"
    
    
    G = parse_xml_and_create_graph(file_path, with_weights=False)
    mapping = dict(zip(G.nodes, random.sample(range(len(G.nodes)), len(G.nodes))))
    G_permuted = nx.relabel_nodes(G, mapping)
    G = G_permuted.subgraph({i for i in range(130)} )
    
    partitions = partition_vertices(G, k)

    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 2, total = True)
    solutions_len2_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=2: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len2_k5_TUC.append(0)
        print(f"TU CORE, max_len=2: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 2)
        solutions_len2_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=2: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2, total = True)
        solutions_len2_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors == 0:
            solutions_len2_k5_WC.append(0)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2)
            solutions_len2_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")
    
    G = parse_xml_and_create_graph(file_path, with_weights=False)
    mapping = dict(zip(G.nodes, random.sample(range(len(G.nodes)), len(G.nodes))))
    G = G_permuted.subgraph({i for i in range(100)} )
    
    partitions = partition_vertices(G, k)
    
    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 3, total = True)
    solutions_len3_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=3: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len3_k5_TUC.append(0)
        print(f"TU CORE, max_len=3: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 3)
        solutions_len3_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=3: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3, total = True)
        solutions_len3_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors == 0:
            solutions_len3_k5_WC.append(0)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3)
            solutions_len3_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")

    


TU CORE max_len=2: for instance 0: we need 0  donors.
TU CORE, max_len=2: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 0: we need 0  donors.
TU CORE, max_len=3: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=3: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=3: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=2: for instance 1: we need 0  donors.
TU CORE, max_len=2: for instance 1: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 1: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 1: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 1: we need 0  donors.
TU CORE, max_len=3: for instance 1: we need 0 improvement in constraints.
WEAK CORE, max_len=3: for instance 1: we need 0 donors. STEPS:

In [ ]:
""" k = 8 (5 small, 3 large) , KEP """

In [4]:
solutions_len2_k5_WC = []
solutions_len2_k5_TUC = []
solutions_len3_k5_TUC = []
solutions_len3_k5_WC = []
 # Number of countries
k = 3
l = 5
for num in range(20):
    file_path = f"/Users/User/Downloads/KEP_instances/genxml-{num}.xml"
    
    
    G = parse_xml_and_create_graph(file_path, with_weights=False)
    mapping = dict(zip(G.nodes, random.sample(range(len(G.nodes)), len(G.nodes))))
    G_permuted = nx.relabel_nodes(G, mapping)
    G = G_permuted.subgraph({i for i in range(130)} )
    
    partitions = partition_vertices_uneven(G, k, l)

    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 2, total = True)
    solutions_len2_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=2: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len2_k5_TUC.append(0)
        print(f"TU CORE, max_len=2: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 2)
        solutions_len2_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=2: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2, total = True)
        solutions_len2_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors == 0:
            solutions_len2_k5_WC.append(0)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2)
            solutions_len2_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")
    
    G = parse_xml_and_create_graph(file_path, with_weights=False)
    mapping = dict(zip(G.nodes, random.sample(range(len(G.nodes)), len(G.nodes))))
    G = G_permuted.subgraph({i for i in range(100)} )
    
    partitions = partition_vertices_uneven(G, k, l)
    
    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 3, total = True)
    solutions_len3_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=3: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len3_k5_TUC.append(0)
        print(f"TU CORE, max_len=3: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 3)
        solutions_len3_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=3: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3, total = True)
        solutions_len3_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors == 0:
            solutions_len3_k5_WC.append(0)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3)
            solutions_len3_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")

    


TU CORE max_len=2: for instance 0: we need 0  donors.
TU CORE, max_len=2: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 0: we need 0  donors.
TU CORE, max_len=3: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=3: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=3: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=2: for instance 1: we need 1  donors.
TU CORE, max_len=2: for instance 1: we need 1 improvement in constraints.
Violations: IR: None, CORE: None, Coalition: None
WEAK CORE, max_len=2: for instance 1: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 1: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 1: we need 0  donors.
TU CORE, max_len=3: for instance 1: we need 0 improvement in constraints.
WEAK CORE, m

C:\Users\User\anaconda3\Lib\site-packages\pulp\pulp.py:1780: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Violations: IR: None, CORE: None, Coalition: None
WEAK CORE, max_len=2: for instance 8: we need 1 donors. STEPS: 1
Violations: IR: 1, CORE: 1, Coalition: (2,)
Violations: IR: None, CORE: None, Coalition: None
WEAK CORE, max_len=2: for instance 8: we need 1 donors per country. STEPS: 1
TU CORE max_len=3: for instance 8: we need 1  donors.
TU CORE, max_len=3: for instance 8: we need 1 improvement in constraints.
Violations: IR: None, CORE: None, Coalition: None
WEAK CORE, max_len=3: for instance 8: we need 0 donors. STEPS: 1
WEAK CORE, max_len=3: for instance 8: we need 0 donors per country. STEPS: 1
TU CORE max_len=2: for instance 9: we need 0  donors.
TU CORE, max_len=2: for instance 9: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 9: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 9: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 9: we need 1  donors.
TU CORE, max_len=3: for instance 9: we need 1 improvement in constrain

In [ ]:
""" k = 9 (7 small, 2 large), KEP  """

In [5]:
solutions_len2_k5_WC = []
solutions_len2_k5_TUC = []
solutions_len3_k5_TUC = []
solutions_len3_k5_WC = []
 # Number of countries
k = 2
l = 7
for num in range(20):
    file_path = f"/Users/User/Downloads/KEP_instances/genxml-{num}.xml"
    
    
    G = parse_xml_and_create_graph(file_path, with_weights=False)
    mapping = dict(zip(G.nodes, random.sample(range(len(G.nodes)), len(G.nodes))))
    G_permuted = nx.relabel_nodes(G, mapping)
    G = G_permuted.subgraph({i for i in range(130)} )
    
    partitions = partition_vertices_uneven(G, k, l)

    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 2, total = True)
    solutions_len2_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=2: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len2_k5_TUC.append(0)
        print(f"TU CORE, max_len=2: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 2)
        solutions_len2_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=2: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2, total = True)
        solutions_len2_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors == 0:
            solutions_len2_k5_WC.append(0)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2)
            solutions_len2_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")
    
    G = parse_xml_and_create_graph(file_path, with_weights=False)
    mapping = dict(zip(G.nodes, random.sample(range(len(G.nodes)), len(G.nodes))))
    G = G_permuted.subgraph({i for i in range(100)} )
    
    partitions = partition_vertices_uneven(G, k, l)
    
    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 3, total = True)
    solutions_len3_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=3: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len3_k5_TUC.append(0)
        print(f"TU CORE, max_len=3: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 3)
        solutions_len3_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=3: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3, total = True)
        solutions_len3_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors == 0:
            solutions_len3_k5_WC.append(0)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3)
            solutions_len3_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")

    


TU CORE max_len=2: for instance 0: we need 1  donors.
TU CORE, max_len=2: for instance 0: we need 1 improvement in constraints.
Violations: IR: None, CORE: None, Coalition: None
WEAK CORE, max_len=2: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 0: we need 1  donors.
TU CORE, max_len=3: for instance 0: we need 1 improvement in constraints.
Violations: IR: 0, CORE: 1, Coalition: (0, 7)
Violations: IR: None, CORE: None, Coalition: None
WEAK CORE, max_len=3: for instance 0: we need 1 donors. STEPS: 1
Violations: IR: 0, CORE: 1, Coalition: (0, 7)
Violations: IR: None, CORE: None, Coalition: None
WEAK CORE, max_len=3: for instance 0: we need 1 donors per country. STEPS: 1
TU CORE max_len=2: for instance 1: we need 0  donors.
TU CORE, max_len=2: for instance 1: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 1: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for

KeyboardInterrupt: 

In [3]:
solutions_len2_k5_WC = []
solutions_len2_k5_TUC = []
solutions_len3_k5_TUC = []
solutions_len3_k5_WC = []
 # Number of countries
k = 2
l = 7
for num in range(20):
    file_path = f"/Users/User/Downloads/KEP_instances/genxml-{num}.xml"
    
    
    G = parse_xml_and_create_graph(file_path, with_weights=False)
    mapping = dict(zip(G.nodes, random.sample(range(len(G.nodes)), len(G.nodes))))
    G_permuted = nx.relabel_nodes(G, mapping)
    G = G_permuted.subgraph({i for i in range(130)} )
    
    partitions = partition_vertices_uneven(G, k, l)

    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 2, total = True)
    solutions_len2_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=2: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len2_k5_TUC.append(0)
        print(f"TU CORE, max_len=2: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 2)
        solutions_len2_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=2: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2, total = True)
        solutions_len2_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors == 0:
            solutions_len2_k5_WC.append(0)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2)
            solutions_len2_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")
    
    G = parse_xml_and_create_graph(file_path, with_weights=False)
    mapping = dict(zip(G.nodes, random.sample(range(len(G.nodes)), len(G.nodes))))
    G = G_permuted.subgraph({i for i in range(100)} )
    
    partitions = partition_vertices_uneven(G, k, l)
    
    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 3, total = True)
    solutions_len3_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=3: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len3_k5_TUC.append(0)
        print(f"TU CORE, max_len=3: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 3)
        solutions_len3_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=3: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3, total = True)
        solutions_len3_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors == 0:
            solutions_len3_k5_WC.append(0)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3)
            solutions_len3_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")

    


TU CORE max_len=2: for instance 0: we need 1  donors.
TU CORE, max_len=2: for instance 0: we need 1 improvement in constraints.
Violations: IR: None, CORE: None, Coalition: None
WEAK CORE, max_len=2: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 0: we need 1  donors.
TU CORE, max_len=3: for instance 0: we need 1 improvement in constraints.
Violations: IR: None, CORE: None, Coalition: None
WEAK CORE, max_len=3: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=3: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=2: for instance 1: we need 0  donors.
TU CORE, max_len=2: for instance 1: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 1: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 1: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 1: we need 1  donors.
TU CORE, max_len=3: for instance 1: 

KeyboardInterrupt: 

In [ ]:
""" k = 6 , 13-17 types """

In [10]:
solutions_len2_k5_WC = []
solutions_len2_k5_TUC = []
solutions_len3_k5_TUC = []
solutions_len3_k5_WC = []

k = 6  # Number of countries

for num in range(20):
    G = generate_typed_graph(150,13+num%4)
    k = 6
    partitions = partition_vertices(G, k)

    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 2, total = True)
    solutions_len2_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=2: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len2_k5_TUC.append(0)
        print(f"TU CORE, max_len=2: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 2)
        solutions_len2_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=2: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2, total = True)
        solutions_len2_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors <= 1:
            solutions_len2_k5_WC.append(0)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2)
            solutions_len2_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")
    
   
    
    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 3, total = True)
    solutions_len3_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=3: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len3_k5_TUC.append(0)
        print(f"TU CORE, max_len=3: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 3)
        solutions_len3_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=3: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3, total = True)
        solutions_len3_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors <= 1:
            solutions_len3_k5_WC.append(0)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3)
            solutions_len3_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")

    


TU CORE max_len=2: for instance 0: we need 0  donors.
TU CORE, max_len=2: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 0: we need 0  donors.
TU CORE, max_len=3: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=3: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=3: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=2: for instance 1: we need 0  donors.
TU CORE, max_len=2: for instance 1: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 1: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 1: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 1: we need 0  donors.
TU CORE, max_len=3: for instance 1: we need 0 improvement in constraints.
WEAK CORE, max_len=3: for instance 1: we need 0 donors. STEPS:

In [ ]:
""" k = 5, varying types """

In [14]:
solutions_len2_k5_WC = []
solutions_len2_k5_TUC = []
solutions_len3_k5_TUC = []
solutions_len3_k5_WC = []

k = 5  # Number of countries

for num in range(20):
    G = generate_typed_graph(150,13+num%4)
    k = 6
    partitions = partition_vertices(G, k)

    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 2, total = True)
    solutions_len2_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=2: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len2_k5_TUC.append(0)
        print(f"TU CORE, max_len=2: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 2)
        solutions_len2_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=2: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2, total = True)
        solutions_len2_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors <= 1:
            solutions_len2_k5_WC.append(0)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2)
            solutions_len2_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")
    
   
    
    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 3, total = True)
    solutions_len3_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=3: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len3_k5_TUC.append(0)
        print(f"TU CORE, max_len=3: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 3)
        solutions_len3_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=3: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3, total = True)
        solutions_len3_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors <= 1:
            solutions_len3_k5_WC.append(0)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3)
            solutions_len3_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")

    


TU CORE max_len=2: for instance 0: we need 0  donors.
TU CORE, max_len=2: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 0: we need 0  donors.
TU CORE, max_len=3: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=3: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=3: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=2: for instance 1: we need 0  donors.
TU CORE, max_len=2: for instance 1: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 1: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 1: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 1: we need 0  donors.
TU CORE, max_len=3: for instance 1: we need 0 improvement in constraints.
WEAK CORE, max_len=3: for instance 1: we need 0 donors. STEPS:

In [ ]:
""" k = 4, varying types """

In [15]:
solutions_len2_k5_WC = []
solutions_len2_k5_TUC = []
solutions_len3_k5_TUC = []
solutions_len3_k5_WC = []

k = 4  # Number of countries

for num in range(20):
    G = generate_typed_graph(150,13+num%4)
    k = 6
    partitions = partition_vertices(G, k)

    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 2, total = True)
    solutions_len2_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=2: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len2_k5_TUC.append(0)
        print(f"TU CORE, max_len=2: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 2)
        solutions_len2_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=2: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2, total = True)
        solutions_len2_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors <= 1:
            solutions_len2_k5_WC.append(0)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2)
            solutions_len2_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")
    
   
    
    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 3, total = True)
    solutions_len3_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=3: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len3_k5_TUC.append(0)
        print(f"TU CORE, max_len=3: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 3)
        solutions_len3_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=3: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3, total = True)
        solutions_len3_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors <= 1:
            solutions_len3_k5_WC.append(0)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3)
            solutions_len3_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")

    


TU CORE max_len=2: for instance 0: we need 0  donors.
TU CORE, max_len=2: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 0: we need 0  donors.
TU CORE, max_len=3: for instance 0: we need 0 improvement in constraints.
WEAK CORE, max_len=3: for instance 0: we need 0 donors. STEPS: 1
WEAK CORE, max_len=3: for instance 0: we need 0 donors per country. STEPS: 1
TU CORE max_len=2: for instance 1: we need 0  donors.
TU CORE, max_len=2: for instance 1: we need 0 improvement in constraints.
WEAK CORE, max_len=2: for instance 1: we need 0 donors. STEPS: 1
WEAK CORE, max_len=2: for instance 1: we need 0 donors per country. STEPS: 1
TU CORE max_len=3: for instance 1: we need 0  donors.
TU CORE, max_len=3: for instance 1: we need 0 improvement in constraints.
WEAK CORE, max_len=3: for instance 1: we need 0 donors. STEPS:

In [ ]:
""" k = 8 (5 small, 3 large) , types """

In [ ]:
solutions_len2_k5_WC = []
solutions_len2_k5_TUC = []
solutions_len3_k5_TUC = []
solutions_len3_k5_WC = []

k = 3  # Number of countries
l = 5
for num in range(20):
    G = generate_typed_graph(150,13+num%4)
    k = 6
    partitions = partition_vertices_uneven(G, k ,l)

    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 2, total = True)
    solutions_len2_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=2: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len2_k5_TUC.append(0)
        print(f"TU CORE, max_len=2: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 2)
        solutions_len2_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=2: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2, total = True)
        solutions_len2_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors <= 1:
            solutions_len2_k5_WC.append(0)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2)
            solutions_len2_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=2: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")
    
   
    
    solution, total_donors = EXACT_near_feasible_TU_core(G, partitions, max_len = 3, total = True)
    solutions_len3_k5_TUC.append(total_donors)
    print(f"TU CORE max_len=3: for instance {num}: we need {total_donors}  donors.")

    if total_donors == 0:
        solutions_len3_k5_TUC.append(0)
        print(f"TU CORE, max_len=3: for instance {num}: we need {total_donors} improvement in constraints.")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: 1")
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: 1")
    else:
        solution, improvement = EXACT_near_feasible_TU_core(G, partitions, max_len = 3)
        solutions_len3_k5_TUC.append(improvement)
        print(f"TU CORE, max_len=3: for instance {num}: we need {improvement} improvement in constraints.")
        
        solution, total_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3, total = True)
        solutions_len3_k5_WC.append(total_donors)
        print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors. STEPS: {i}")
        if total_donors <= 1:
            solutions_len3_k5_WC.append(0)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {total_donors} donors per country. STEPS: {i}")
        else:
            solution, local_donors, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 3)
            solutions_len3_k5_WC.append(local_donors)
            print(f"WEAK CORE, max_len=3: for instance {num}: we need {local_donors} donors per country. STEPS: {i}")

    


In [ ]:
""" OTHERS SIMULATIONS """

In [27]:
solutions = []

for num in range(20):
    G = generate_typed_graph(100,15)
    k = 7
    partitions = partition_vertices(G, k)
    
    solution, improvement, i = near_feasible_weak_core_heuristic(G, partitions, max_len = 2)
    solutions.append((solution,improvement))
    print(f"WEAK CORE: for instance {num}: we need {improvement} wiggle room. STEPS: {i}")


Violations: IR: None, CORE: None
WEAK CORE: for instance 0: we need 0 wiggle room. STEPS: 1
Violations: IR: None, CORE: None
WEAK CORE: for instance 1: we need 0 wiggle room. STEPS: 1
Violations: IR: None, CORE: None
WEAK CORE: for instance 2: we need 0 wiggle room. STEPS: 1
Violations: IR: None, CORE: None
WEAK CORE: for instance 3: we need 0 wiggle room. STEPS: 1
Violations: IR: None, CORE: None
WEAK CORE: for instance 4: we need 0 wiggle room. STEPS: 1
Violations: IR: None, CORE: None
WEAK CORE: for instance 5: we need 0 wiggle room. STEPS: 1
Violations: IR: 1, CORE: 1


C:\Users\User\anaconda3\Lib\site-packages\pulp\pulp.py:1780: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


WEAK CORE: for instance 6: we need 0 wiggle room. STEPS: 2
Violations: IR: 1, CORE: 1
WEAK CORE: for instance 7: we need 0 wiggle room. STEPS: 2
Violations: IR: 1, CORE: 1
WEAK CORE: for instance 8: we need 0 wiggle room. STEPS: 3
Violations: IR: 1, CORE: 1
WEAK CORE: for instance 9: we need 0 wiggle room. STEPS: 9
Violations: IR: 1, CORE: 1
Violations: IR: None, CORE: None
WEAK CORE: for instance 10: we need 1 wiggle room. STEPS: 1
Violations: IR: 1, CORE: 1
WEAK CORE: for instance 11: we need 0 wiggle room. STEPS: 4
Violations: IR: None, CORE: None
WEAK CORE: for instance 12: we need 0 wiggle room. STEPS: 1
Violations: IR: None, CORE: None
WEAK CORE: for instance 13: we need 0 wiggle room. STEPS: 1
Violations: IR: None, CORE: None
WEAK CORE: for instance 14: we need 0 wiggle room. STEPS: 1
Violations: IR: 1, CORE: 1
Violations: IR: None, CORE: None
WEAK CORE: for instance 15: we need 1 wiggle room. STEPS: 1
Violations: IR: None, CORE: None
WEAK CORE: for instance 16: we need 0 wiggle

In [7]:
UG = generate_random_graph(100,0.01)
M = find_maximum_matching(UG)
print(len(M))
k = 5
partitions = partition_vertices(UG, k)

check_core_stability(UG, M, partitions)
check_weak_core_stability(UG, M, partitions)


Maximum Matching: {(77, 75), (94, 3), (98, 27), (57, 17), (99, 90), (45, 0), (78, 67), (49, 37), (23, 12), (59, 14), (93, 64), (76, 60), (96, 2), (18, 43), (6, 47), (85, 69), (68, 4), (89, 36), (39, 1), (92, 44), (79, 10), (46, 31), (82, 30), (72, 19), (88, 40), (62, 33), (97, 24), (61, 13), (29, 11)}
29
Matching is in the core.


'Matching is in the weak core.'

In [7]:
#NO-INSTANCE CHECK

import networkx as nx
import random

# Create the graph
G = nx.Graph()

# Add two triangles
triangle_1 = [0, 1, 2]
triangle_2 = [3, 4, 5]
G.add_edges_from([(triangle_1[i], triangle_1[j]) for i in range(3) for j in range(i+1, 3)])
G.add_edges_from([(triangle_2[i], triangle_2[j]) for i in range(3) for j in range(i+1, 3)])

# Add three K_5 graphs
K5_1 = [6, 7, 8, 9, 10]
K5_2 = [11, 12, 13, 14, 15]
K5_3 = [16, 17, 18, 19, 20]

for K5 in [K5_1, K5_2, K5_3]:
    G.add_edges_from([(K5[i], K5[j]) for i in range(5) for j in range(i+1, 5)])

# Assign vertices to 3 countries
countries = {0: 0, 1: 1, 2: 2, 3: 0, 4: 1, 5: 2}  # One from each triangle

# Assign K5 partitions
def partition_K5(K5, start_country):
    """Distribute vertices in K5 such that two countries get 2 and one gets 1."""
    country_assignment = [start_country, (start_country + 1) % 3, (start_country + 2) % 3,
                          start_country, (start_country + 1) % 3]
    random.shuffle(country_assignment)  # Shuffle to randomize distribution
    return {K5[i]: country_assignment[i] for i in range(5)}

for i, K5 in enumerate([K5_1, K5_2, K5_3]):
    countries.update(partition_K5(K5, i))  # Cycle start country

# Print the partition
print("Partition:", countries)

# Visualize the graph with colors for countries
import matplotlib.pyplot as plt

colors = ['red', 'blue', 'green']
node_colors = [colors[countries[v]] for v in G.nodes()]

#nx.draw(G, with_labels=True, node_color=node_colors, node_size=500, edge_color='gray')

M = find_maximum_matching(G)
#check_weak_core_stability(G, M, countries)

G = to_directed(G)

Partition: {0: 0, 1: 1, 2: 2, 3: 0, 4: 1, 5: 2, 6: 0, 7: 1, 8: 1, 9: 2, 10: 0, 11: 2, 12: 1, 13: 2, 14: 0, 15: 1, 16: 2, 17: 0, 18: 2, 19: 1, 20: 0}
Maximum Matching: {(15, 11), (19, 17), (5, 3), (2, 0), (20, 16), (10, 6), (14, 12), (9, 7)}


In [5]:
solution, total_donors = EXACT_near_feasible_TU_core(G, countries, max_len = 2, total = True)

print(f"we need {total_donors} donors.")
solution, improvement = EXACT_near_feasible_TU_core(G, countries, max_len = 2)

print(f"we need {improvement} slack per constraint.")

we need 2 donors.
we need 2 slack per constraint.


In [8]:
sol, total_donors, i = near_feasible_weak_core_heuristic(to_directed(G), countries, max_len = 2, total = True)
print(f"we need {total_donors} donors. STEPS {i}")

sol, local_donors, i = near_feasible_weak_core_heuristic(to_directed(G), countries, max_len = 2)
print(f"we need {local_donors} donors per country. STEPS {i}")


Violations: IR: 0, CORE: 1, Coalition: (0, 2)
Violations: IR: 0, CORE: 1, Coalition: (1, 2)
we need 1 donors. STEPS 2
Violations: IR: 0, CORE: 1, Coalition: (0, 2)
Violations: IR: None, CORE: None, Coalition: None
we need 1 donors per country. STEPS 1


In [ ]:
num = 6
file_path = f"/Users/User/Downloads/KEP_instances/genxml-{num}.xml"
G = parse_xml_and_create_graph(file_path, with_weights=False)
C_opt = find_maxL_packing(G)
print(f"#transplants in unbounded: {len(C_opt)}")
for i in range(2,8):
    C = find_maxL_packing(G,i)
    print(f"#transplants with bound {i}: {len(C)}")
    if len(C) == len(C_opt):
        break

#transplants in unbounded: 45
#transplants with bound 2: 18
#transplants with bound 3: 34
